In [1]:
# install dependencies
!pip install jupyter-dash -q
!pip install dash-bootstrap-components
!pip install pystan==2.19.1.1
!pip install prophet

In [2]:
# import dependencies
from fbprophet import Prophet
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
sns.set_theme()

In [3]:
# import data
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
%cd /content/gdrive/MyDrive/Colab Notebooks/Portfolios/forecast

/content/gdrive/MyDrive/Colab Notebooks/Portfolios/forecast


In [5]:
data = pd.read_csv('ecommerce_dataset.csv')

In [6]:
# drop unnamed columns
data.drop(columns=['Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25'], inplace=True)

In [7]:
# check quick distribution data
data.describe()

,item_id,price,qty_ordered,grand_total,discount_amount,Year,Month,Customer ID
count,584524.000000,5.845240e+05,584524.000000,5.845240e+05,584524.000000,584524.000000,584524.000000,584513.000000
mean,565667.074218,6.348748e+03,1.296388,8.530619e+03,499.492775,2017.044115,7.167654,45790.511965
std,200121.173648,1.494927e+04,3.996061,6.132081e+04,1506.943046,0.707355,3.486305,34414.962389
min,211131.000000,0.000000e+00,1.000000,-1.594000e+03,-599.500000,2016.000000,1.000000,1.000000
25%,395000.750000,3.600000e+02,1.000000,9.450000e+02,0.000000,2017.000000,4.000000,13516.000000
50%,568424.500000,8.990000e+02,1.000000,1.960400e+03,0.000000,2017.000000,7.000000,42856.000000
75%,739106.250000,4.070000e+03,1.000000,6.999000e+03,160.500000,2018.000000,11.000000,73536.000000
max,905208.000000,1.012626e+06,1000.000000,1.788800e+07,90300.000000,2018.000000,12.000000,115326.000000


In [8]:
# because grand_total can't under 0, we must exclude data grand_total under 0
data = data[data.grand_total > 0]

In [9]:
# check null data
data.isnull().sum()

item_id                       0
status                       15
created_at                    0
sku                           5
price                         0
qty_ordered                   0
grand_total                   0
increment_id                  0
category_name_1             163
sales_commission_code    134206
discount_amount               0
payment_method                0
Working Date                  0
BI Status                     0
 MV                           0
Year                          0
Month                         0
Customer Since               11
M-Y                           0
FY                            0
Customer ID                  11
dtype: int64

In [10]:
# drop null value
data = data.dropna()

In [11]:
# rename some columns
data.rename(columns={
    'category_name_1':'category_name',
    'Working Date':'Working_Date',
    ' MV ':'MV',
    'Customer Since':'Customer_Since',
    'Customer ID':'Customer_ID'
}, inplace=True)

In [12]:
# becase there are a lot of payment status, so we must simplify it.
data.status = data.status.replace({
    'complete':'Completed',
    'received':'Completed',
    'cod':'Completed',
    'paid':'Completed',
    'closed':'Completed',
    'exchange':'Completed',
    'canceled':'Canceled',
    'order_refunded':'Canceled',
    'refund':'Canceled',
    'fraud':'Canceled',
    '\\N':'Canceled',
    'payment_review':'Pending',
    'pending':'Pending',
    'processing':'Pending',
    'holded':'Pending',
    'pending_paypal':'Pending'
})

In [13]:
# create dataframe copy and filter status = Completed
data_copy = data[data.status == 'Completed']

In [14]:
# change data format
data_copy.created_at = pd.to_datetime(data_copy.created_at)

In [15]:
# export month and year transaction date 
data_copy['month_year'] = pd.to_datetime(data_copy.created_at.dt.strftime('%Y-%m'))

In [16]:
# grouping transaction each month
data_grouping = data_copy.groupby(['month_year'])['grand_total'].sum()
data_grouping = pd.DataFrame(data_grouping).reset_index()

In [17]:
# sort data based on month_year ascending
data_grouping.sort_values(['month_year'], inplace=True)

# Forecasting data for next 30 days.

In [18]:
data_grouping.columns

Index(['month_year', 'grand_total'], dtype='object')

In [19]:
m = Prophet()
m.fit(data_grouping.rename(columns={"month_year":"ds", "grand_total":"y"}))
future = m.make_future_dataframe(periods=30)
forecast = m.predict(future)

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 19.


In [21]:
# get forecast data, rename columns and add flag section columns
future_forecast = forecast.iloc[-30:][['ds', 'yhat']]
future_forecast.columns = ["Date", "Total"]
future_forecast['section'] = "Forecast"

# add flag sction for current data
data_grouping['section'] = "History"

In [30]:
# rename data_grouping columns
data_grouping.rename(columns={
    'month_year':'Date',
    'grand_total':'Total'
}, inplace=True)

In [32]:
# join 2 dataframe
df_plot = pd.concat([data_grouping, future_forecast], axis=0)

In [37]:
# visualize Data
import plotly.express as px
px.line(df_plot, x='Date', y='Total', color='section')